In [1]:
from IPython.display import Image

In [2]:
Image(url='https://i.imgur.com/S7KH5hZ.png', width=600)

## Training

$$
\nabla_\theta\left\|\epsilon - \epsilon_\theta(\sqrt{\bar\alpha_t}x_0+\sqrt{1-\bar\alpha_t}\epsilon, t)\right\|^2
$$

- $\epsilon$: target noise
- $\sqrt{\bar\alpha_t}x_0+\sqrt{1-\bar\alpha_t}\epsilon$: noisy image
- $\epsilon_\theta(\cdot, t)$: noise predictor
    - $\epsilon_\theta(, t)$：neural Network
        - unet

## Sampling

$$
x_{t-1}=\frac{1}{\sqrt{\alpha_t}}\left(x_t-\frac{1-\alpha_t}{\sqrt{1-\bar{\alpha_t}}}\epsilon_\theta(x_t,t)\right)+\sigma_tz
$$

- $\epsilon_\theta(x_t, t)$ unet denoising model
    - input: $(x_t, t)$
- $\sigma_tz$：为noise
- 抛开这个 noise 不谈，

$$
x_{t-1}=\frac{1}{\sqrt{\alpha_t}}\left(x_t-\frac{1-\alpha_t}{\sqrt{1-\bar{\alpha_t}}}\epsilon_\theta(x_t,t)\right)\\
\sqrt{\alpha_t}x_{t-1}+\frac{1-\alpha_t}{\sqrt{1-\bar\alpha_t}}\epsilon_\theta(x_t,t)=x_t
$$